In [1]:
from Executor import *

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [42]:
# Reset the model, and run same finetuning using precomputed conv. model outputs. See if they match...
executor = ExecutorBuilder().\
    with_runID("second").\
    and_().\
    with_Vgg16().\
    and_().\
    train_batch_size(128). \
    and_(). \
    learn_rate(0.001).\
    and_().\
    data_on_path("../data/full/").\
    build()

Pretrained Vgg16 model loaded.
Found 6255 images belonging to 3 classes.
Found 1549 images belonging to 3 classes.
initialized training data from: ../data/full/train
initialized validation data from: ../data/full/valid
found number of softmax classes: 3


In [4]:
executor.precompute_conv_model_outputs()

precomputing conv. model outputs..
Found 6255 images belonging to 3 classes.
Found 1549 images belonging to 3 classes.
('models saved to files: ', 'precomputed_trn_features.second.h5', ' and ', 'precomputed_val_features.second.h5')
done.


<Executor.Executor instance at 0x7f3f4fbc9560>

In [3]:
executor.load_precomputed_conv_models()

loading precomputed conv. outputs...
done...


<Executor.Executor instance at 0x7feec6931368>

In [5]:
fc_model = executor.get_rescaled_fc_model(new_dropout=0.5)

getting rescaled fc model...
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
done...
('updating dropout from: ', 0.5, ' to: ', 0.5)


In [7]:
for layer in fc_model.layers:
    layer.trainable = False

fc_model.layers[-1].trainable = True

In [8]:
opt = Adam(lr=0.001)
fc_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=5,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

fine tuning the rescaled fc model...
Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 3s - loss: 1.7553 - acc: 0.4505 - val_loss: 1.0204 - val_acc: 0.4945
Epoch 2/5
6255/6255 [==============================] - 3s - loss: 1.4248 - acc: 0.4946 - val_loss: 0.9579 - val_acc: 0.5552
Epoch 3/5
6255/6255 [==============================] - 3s - loss: 1.2891 - acc: 0.4964 - val_loss: 0.9186 - val_acc: 0.5358
Epoch 4/5
6255/6255 [==============================] - 3s - loss: 1.1594 - acc: 0.5194 - val_loss: 0.9145 - val_acc: 0.5604
Epoch 5/5
6255/6255 [==============================] - 3s - loss: 1.0779 - acc: 0.5257 - val_loss: 0.9218 - val_acc: 0.5449


In [9]:
fc_model.save_weights("linear_model.second.weights.h5")

In [10]:
opt = Adam(lr=0.0001)
fc_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=5,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

fine tuning the rescaled fc model...
Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 3s - loss: 1.0277 - acc: 0.5437 - val_loss: 0.8684 - val_acc: 0.5778
Epoch 2/5
6255/6255 [==============================] - 3s - loss: 0.9952 - acc: 0.5624 - val_loss: 0.8628 - val_acc: 0.5726
Epoch 3/5
6255/6255 [==============================] - 3s - loss: 0.9945 - acc: 0.5535 - val_loss: 0.8626 - val_acc: 0.5855
Epoch 4/5
6255/6255 [==============================] - 3s - loss: 0.9855 - acc: 0.5520 - val_loss: 0.8603 - val_acc: 0.5810
Epoch 5/5
6255/6255 [==============================] - 3s - loss: 0.9539 - acc: 0.5674 - val_loss: 0.8577 - val_acc: 0.5830


In [11]:
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=5,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 3s - loss: 0.9714 - acc: 0.5557 - val_loss: 0.8525 - val_acc: 0.5823
Epoch 2/5
6255/6255 [==============================] - 3s - loss: 0.9622 - acc: 0.5573 - val_loss: 0.8537 - val_acc: 0.5849
Epoch 3/5
6255/6255 [==============================] - 3s - loss: 0.9520 - acc: 0.5682 - val_loss: 0.8516 - val_acc: 0.5842
Epoch 4/5
6255/6255 [==============================] - 3s - loss: 0.9285 - acc: 0.5755 - val_loss: 0.8504 - val_acc: 0.5978
Epoch 5/5
6255/6255 [==============================] - 3s - loss: 0.9292 - acc: 0.5778 - val_loss: 0.8529 - val_acc: 0.5933


In [12]:
for layer in fc_model.layers:
    layer.trainable = True

In [13]:
fc_model.save_weights("linear_model_veryfine.second.weights.h5")

In [15]:
fc_model.load_weights("linear_model_veryfine.second.weights.h5")

In [18]:
for layer in fc_model.layers:
    layer.trainable = False

fc_model.layers[-1].trainable=True
fc_model.layers[-3].trainable=True

In [35]:
fc_model.load_weights("linear_model_veryfine.second.weights.h5")

In [36]:
opt = Adam(lr=0.0001)
fc_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=3,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

fine tuning the rescaled fc model...
Train on 6255 samples, validate on 1549 samples
Epoch 1/3
6255/6255 [==============================] - 7s - loss: 0.9520 - acc: 0.5429 - val_loss: 0.8754 - val_acc: 0.5720
Epoch 2/3
6255/6255 [==============================] - 7s - loss: 0.7319 - acc: 0.6617 - val_loss: 0.7377 - val_acc: 0.6649
Epoch 3/3
6255/6255 [==============================] - 7s - loss: 0.5320 - acc: 0.7709 - val_loss: 0.7748 - val_acc: 0.6708


In [37]:
print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=3,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

fine tuning the rescaled fc model...
Train on 6255 samples, validate on 1549 samples
Epoch 1/3
6255/6255 [==============================] - 7s - loss: 0.3196 - acc: 0.8708 - val_loss: 0.8553 - val_acc: 0.7069
Epoch 2/3
6255/6255 [==============================] - 7s - loss: 0.1917 - acc: 0.9290 - val_loss: 1.0775 - val_acc: 0.6811
Epoch 3/3
6255/6255 [==============================] - 7s - loss: 0.1432 - acc: 0.9429 - val_loss: 1.0021 - val_acc: 0.7005


In [38]:
fc_model.save_weights("linear_2dense_model_veryfine.second.weights.h5")

In [33]:
print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=2,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

fine tuning the rescaled fc model...
Train on 6255 samples, validate on 1549 samples
Epoch 1/2
6255/6255 [==============================] - 7s - loss: 0.0939 - acc: 0.9741 - val_loss: 0.6969 - val_acc: 0.7353
Epoch 2/2
6255/6255 [==============================] - 7s - loss: 0.0732 - acc: 0.9826 - val_loss: 0.7204 - val_acc: 0.7334


In [39]:
for layer in fc_model.layers:
    layer.trainable = True

In [40]:
opt = Adam(lr=0.00001)
fc_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=3,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

fine tuning the rescaled fc model...
Train on 6255 samples, validate on 1549 samples
Epoch 1/3
6255/6255 [==============================] - 7s - loss: 0.0611 - acc: 0.9783 - val_loss: 1.1283 - val_acc: 0.7211
Epoch 2/3
6255/6255 [==============================] - 7s - loss: 0.0300 - acc: 0.9920 - val_loss: 1.1884 - val_acc: 0.7243
Epoch 3/3
6255/6255 [==============================] - 7s - loss: 0.0211 - acc: 0.9934 - val_loss: 1.2676 - val_acc: 0.7256


In [29]:
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=2,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

Train on 6255 samples, validate on 1549 samples
Epoch 1/2
6255/6255 [==============================] - 7s - loss: 0.1583 - acc: 0.9514 - val_loss: 0.6715 - val_acc: 0.7218
Epoch 2/2
6255/6255 [==============================] - 7s - loss: 0.1160 - acc: 0.9699 - val_loss: 0.6895 - val_acc: 0.7405


In [43]:
executor.init_vgg_with_retrained_fc_layers(fc_model)

<Executor.Executor instance at 0x7f3f33f682d8>

In [44]:
executor.save_model_to_file()

<Executor.Executor instance at 0x7f3f33f682d8>

In [45]:
executor.build_predictions_on_test_data().and_().save_predictions_to_file("best")

Found 512 images belonging to 1 classes.


/home/ubuntu/kaggle_cervical_cancer_code/src/best